## Example of MCP (Model Context Protocol) call from a ReAct Agent

Adapted from : https://github.com/rectalogic/langchain-mcp

In [22]:
# Default setup cell.
import builtins

from devtools import debug
from dotenv import load_dotenv

setattr(builtins, "debug", debug)
load_dotenv(verbose=True)
%load_ext autoreload
%autoreload 2
%reset -f
!export PYTHONPATH=":./python"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import pathlib

from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.tools import BaseTool
from langchain_mcp.toolkit import MCPToolkit
from langgraph.prebuilt import create_react_agent
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from python.ai_core.llm import get_llm

# set_debug(True)
# set_verbose(True)

MODEL_ID = "llama31_8_groq"
llm = get_llm(llm_id=MODEL_ID)

In [ ]:
async def run(llm: BaseChatModel, tools: list[BaseTool], prompt: str):
    agent = create_react_agent(llm, tools=tools)
    inputs = {"messages": [("user", prompt)]}
    return await agent.ainvoke(inputs)


async def mcp_file_system(llm: BaseChatModel, prompt: str):
    server_params = StdioServerParameters(
        command="npx",
        args=["-y", "@modelcontextprotocol/server-filesystem", str(pathlib.Path.cwd().parent.parent)],
    )
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            toolkit = MCPToolkit(session=session)
            await toolkit.initialize()
            return await run(llm, toolkit.get_tools(), prompt)


In [ ]:
prompt = "list parent directory of ."
r = await mcp_file_system(llm, prompt=prompt)
message = r["messages"][-1]
debug(message.pretty_print())